# Wendy Johansson: Co-Founder & Chief Product Experience Officer, MiSalud Health

### “We told [people in the supermarket], ‘You can get free consultations with Spanish-speaking doctors here and now, right on your phone, you don't need insurance, you don’t need to prefill any personal information. During those situations we learned a couple of things we already knew about, but really got to get personal with people about, ‘Well, can I sign up if I don't have an ID,’ and we’re like, ‘What do you mean you don't have an ID?’ Basically people started intimating that they're undocumented.” —via [Awkward Silences](https://podcast.userinterviews.com/episodes/99-leading-ux-research-for-healthcare-apps-with-wendy-johansson-of-misalud-health/transcript)

### With Wendy, we examine how AI can be used to reshape one's learning experiences in health in an adaptive manner to avoid the "Phase 1 Limbo" problem in an existing native code app.

## 🔥 Let's get the required packages and fire up a kernel

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.1.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: YamlDotNet, 13.7.1"

In [ ]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## ⛩️ I always need to do some JSON warmup exercises

Note that there's a fancy "JSON Mode" with the OAI/AOAI API but I'm not using it here ...

### Let's cook a list of strings in JSON format

In [ ]:
var result = await kernel.InvokePromptAsync("Output a list of three fruits as a plain JSON list please without including ```json``` in the output.");
Console.WriteLine(Utils.WordWrap(result.ToString(), 80));

### Let's bring that freshly cooked string into the native code world via JSON teleportation

In [ ]:
List<string> fruits = JsonSerializer.Deserialize<List<string>>(result.ToString());

foreach (var fruit in fruits)
{
    Console.WriteLine(fruit);
}

### Great! And now let's reconstitute it as a conventional string again to go pure semantic

In [ ]:
string jsonOutput = JsonSerializer.Serialize(fruits);

Console.WriteLine(jsonOutput);

### Voila! You've mastered basic JSON to/from the semantic/native universe

If you want to use the more fancy "JSON mode" in OAI/AOAI, here's an example below. Note that the caveat is you need to be sure to include the magic word "JSON" in your prompt or [this won't work](https://community.openai.com/t/how-do-i-use-the-new-json-mode/475890).

In [ ]:
#pragma warning disable SKEXP0013

KernelArguments args = new(new OpenAIPromptExecutionSettings { ResponseFormat = "json_object" }) { { "topic", "chocolate" } };
Console.WriteLine(await kernel.InvokePromptAsync("Create a recipe for a {{$topic}} cake in JSON format", args));

## 🧑‍🍳 Let's get cooking on this problem together!

### 🔥 We fire up Semantic Kernel's Experimental Agents ...

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.1.0-alpha"
#r "nuget: YamlDotNet, 13.7.1"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

public class Agenty
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

List<string> agentNames = ["LessonPlanner"];
string team = "";
int i = 1;
List<(string Name, string Instructions)> agentInfo = new();

foreach(var a in agentNames)
{
    var yaml = File.ReadAllText($"../agents/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();

    var p = deserializer.Deserialize<Agenty>(yaml);
    Console.WriteLine($"Agent: {a}");
    team += $"{i}) {p.Name}: {p.Description}, ";
    i++;
    agentInfo.Add((p.Name, p.Instructions));
}

agentInfo

### 🧑‍🍳 Let's craft a lesson plan

In [ ]:
static string lessonPlan = @"Your lesson plan spans three weeks:

1/ Why health matters and measuring it regularly
2/ What the health data is useful for to monitor your diabetes condition
3/ How to use the recommendations compiled over the last three weeks.

In the first week you learn how to measure your temperature and blood pressure and to record it in the app. 

In the second week you learn why this data is important. 

In the third week you learn how the accumulated data can actually serve you well.";

### 🧑‍🍳 And instantiate the agent(s)

In [ ]:
#pragma warning disable SKEXP0101

static readonly List<IAgent> s_agents = new();

async Task<IAgent> CreateAgentAsync(string name, string jobDescription)
{
    return Track(
        await new AgentBuilder()
            .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
            .WithInstructions(jobDescription)
            .WithName(name)
            .WithDescription(name)
            .BuildAsync());
}

IAgent Track(IAgent agent)
{
    s_agents.Add(agent);

    return agent;
}

var agents = new List<IAgent>();
foreach (var (name, jobDescription) in agentInfo)
{
    var agent = await CreateAgentAsync(name, jobDescription);

    agents.Add(agent);
}

agents

### 🧵 We need a thread

In [ ]:
#pragma warning disable SKEXP0101

IAgentThread? thread = null;

thread = await agents[0].NewThreadAsync();
await thread.AddUserMessageAsync($"{lessonPlan}");

### 🕵️💨 And then we're ready to let the agent do its thing

In [ ]:
#pragma warning disable SKEXP0101

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 🎤 Voice is cool to work with, so let's do a bit of that

In [ ]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.34.1"
#r "nuget: NetCoreAudio, 1.8.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0"

#!import ../config/Utils.cs 
#!import ../config/AzureSpeech.cs

Utils.LoadEnvFile();

string subscriptionKey = Environment.GetEnvironmentVariable("AZURE_SPEECH_KEY");
string subscriptionRegion = Environment.GetEnvironmentVariable("AZURE_SPEECH_REGION");

var speechService = new SpeechRecognitionService(subscriptionKey, subscriptionRegion);

### 🎤👩🏻 Speak something (for less than 15 seconds and w/o stopping)

In [ ]:
string recognizedText = await speechService.RecognizeOnceAsync();
recognizedText

### 🎤🧑 Let's use this pattern now as the user checking in

In [ ]:
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"Transcribed: {recognizedText}");

### 👂🕵️ The agent can lend their ear

In [ ]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync($"{recognizedText}");

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 📦 Now you want to send this information to your app

In [ ]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync(@"Provide a succinct summary of the plan 
per week as a JSON list where each item is a string with the format 
'Week {{week number}} / {{title}}  / Complete: {{completion status as done if the week has ended or in progress if it's the current week}}'
Only give the output as the JSON list and do not include ```json``` in the output.");

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 200));

## 👩‍⚕️📖 Let's say you want to change the lesson plan

### 🎤👩‍⚕️ You can change the lesson plan

Your lesson plan spans four weeks:

1. Why health matters
2. What the health data is useful for 
3. How to use the recommendations 
4. Visiting the doctor

In [ ]:
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"Transcribed: {recognizedText}");

### 📦 Bring it back to the native code land

In [ ]:
KernelArguments args = new(new OpenAIPromptExecutionSettings { MaxTokens = 500, Temperature = 0.5 }) { { "input", "nada" } };
var result = await kernel.InvokePromptAsync(
$"{recognizedText} Output this list as a plain JSON list of concise elements with the form 'Week {{week number}} / {{title}}'' please without including ```json``` in the output."
    , args); 
Console.WriteLine(result.ToString());

In [ ]:
List<string> units = JsonSerializer.Deserialize<List<string>>(result.ToString());

foreach (var unit in units)
{
    Console.WriteLine(unit);
}